# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import plotly.express as px

# Import API key
from config import geoapify_key

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,smithers,54.7804,-127.1743,262.33,92,40,0.00,CA,1676392563
1,1,upernavik,72.7868,-56.1549,244.76,92,100,3.64,GL,1676392563
2,2,magomeni,-6.8000,39.2500,301.36,74,20,4.63,TZ,1676392564
3,3,busselton,-33.6500,115.3333,289.18,90,37,3.68,AU,1676392565
4,4,acapulco,16.8634,-99.8901,301.05,74,40,1.54,MX,1676392315


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:


# Create the map
fig = px.scatter_mapbox(city_data_df, lat="Lat", lon="Lng", size="Humidity",
                        color="Humidity", color_continuous_scale="Viridis",
                        size_max=15, zoom=5,
                        mapbox_style="open-street-map")

# Show the map
fig.show()


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities 
filtered_city_data = city_data_df[(city_data_df['Max Temp'] < 27) & 
                                   (city_data_df['Max Temp'] > 21) & 
                                   (city_data_df['Wind Speed'] < 4.5) & 
                                   (city_data_df['Cloudiness'] == 0)]
filtered_city_data
if filtered_city_data.empty:
    print("The filtered DataFrame is empty.")
else:
    print("The filtered DataFrame is not empty.")


The filtered DataFrame is empty.


In [25]:
# Drop any rows with null values
filter_df = city_data_df.dropna(thresh=2)


# Display sample data
filter_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,smithers,54.7804,-127.1743,262.33,92,40,0.00,CA,1676392563
1,1,upernavik,72.7868,-56.1549,244.76,92,100,3.64,GL,1676392563
2,2,magomeni,-6.8000,39.2500,301.36,74,20,4.63,TZ,1676392564
3,3,busselton,-33.6500,115.3333,289.18,90,37,3.68,AU,1676392565
4,4,acapulco,16.8634,-99.8901,301.05,74,40,1.54,MX,1676392315


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filter_df.copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,smithers,54.7804,-127.1743,262.33,92,40,0.00,CA,1676392563,
1,1,upernavik,72.7868,-56.1549,244.76,92,100,3.64,GL,1676392563,
2,2,magomeni,-6.8000,39.2500,301.36,74,20,4.63,TZ,1676392564,
3,3,busselton,-33.6500,115.3333,289.18,90,37,3.68,AU,1676392565,
4,4,acapulco,16.8634,-99.8901,301.05,74,40,1.54,MX,1676392315,
5,5,faya,18.3851,42.4509,289.44,40,50,2.86,SA,1676392566,
6,6,port moresby,-9.4431,147.1797,298.83,83,40,3.60,PG,1676392567,
7,7,ribeira grande,38.5167,-28.7000,289.37,55,20,4.12,PT,1676392567,
8,8,samarai,-10.6167,150.6667,300.66,81,59,5.41,PG,1676392568,
9,9,cape town,-33.9258,18.4232,296.54,70,20,11.83,ZA,1676392516,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius" : 10000,
    "categories": 'accomodation.hotel',
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lattitude = row["Lat"]
    longitude = row["Lng"] 
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{lattitude},{radius}"
    params["bias"] = f"proximity:{longitude},{lattitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
smithers - nearest hotel: No hotel found
upernavik - nearest hotel: No hotel found
magomeni - nearest hotel: No hotel found
busselton - nearest hotel: No hotel found
acapulco - nearest hotel: No hotel found
faya - nearest hotel: No hotel found
port moresby - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
samarai - nearest hotel: No hotel found
cape town - nearest hotel: No hotel found
madang - nearest hotel: No hotel found
srednekolymsk - nearest hotel: No hotel found
novovoronezh - nearest hotel: No hotel found
bambous virieux - nearest hotel: No hotel found
shache - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,smithers,54.7804,-127.1743,262.33,92,40,0.00,CA,1676392563,No hotel found
1,1,upernavik,72.7868,-56.1549,244.76,92,100,3.64,GL,1676392563,No hotel found
2,2,magomeni,-6.8000,39.2500,301.36,74,20,4.63,TZ,1676392564,No hotel found
3,3,busselton,-33.6500,115.3333,289.18,90,37,3.68,AU,1676392565,No hotel found
4,4,acapulco,16.8634,-99.8901,301.05,74,40,1.54,MX,1676392315,No hotel found
5,5,faya,18.3851,42.4509,289.44,40,50,2.86,SA,1676392566,No hotel found
6,6,port moresby,-9.4431,147.1797,298.83,83,40,3.60,PG,1676392567,No hotel found
7,7,ribeira grande,38.5167,-28.7000,289.37,55,20,4.12,PT,1676392567,No hotel found
8,8,samarai,-10.6167,150.6667,300.66,81,59,5.41,PG,1676392568,No hotel found
9,9,cape town,-33.9258,18.4232,296.54,70,20,11.83,ZA,1676392516,No hotel found


In [13]:
### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.
import folium

# Create a map centered on the city
city_map = folium.Map(location=[city_latitude, city_longitude], zoom_start=13)

# Add a marker for each city
for lat, lng, city, hotel, country in zip(city_latitudes, city_longitudes, city_names, hotel_names, countries):
    folium.Marker(
        location=[lat, lng],
        popup=f'City: {city}<br>Hotel: {hotel}<br>Country: {country}',
        tooltip=f'{city}, {country}'
    ).add_to(city_map)

# Display the map
city_map


NameError: name 'city_latitude' is not defined

In [ ]:


# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE